In [133]:
class field(object):
    def __init__(self, massDim, LORindices, ONindices, symbol):
        self.massDim = massDim #int
        self.LORindices = LORindices #list of strings
        self.ONindices = ONindices #list of strings
        self.LORrank = len(LORindices)
        self.ONrank = len(ONindices)
        self.symbol = symbol #string representing field
        self.factors = [] #2-element list of factors that are multiplied or contracted to get this field - i.e., internal structure of field, if any
        self.LORcontract = [] #list of pairs (lists) of contracted Lorentz indices, one from each factor field
        self.ONcontract = [] #list of pairs (lists) of O(N) indices, one from each factor field
        self.tensorRep = ''
        self.elementary = [] #Stores list of N elementary field factors phi and dphi
        self.contractions = [] #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links. 
    def __str__(self): #Prints out attributes of field object
        return 'massDim: ' + str(self.massDim) + '\n' \
            + 'LORindices: ' + str(self.LORindices) + '\n' \
            + 'ONindices: ' + str(self.ONindices) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'LORrank: ' + str(self.LORrank) + '\n' \
            + 'ONrank: ' + str(self.ONrank) + '\n' \
            + 'tensorRep: ' + str(self.tensorRep)
    
    #Define get and set functions
    def get_massDim(self):
        return self.massDim
    def get_LORindices(self):
        return self.LORindices
    def get_ONindices(self):
        return self.ONindices
    def get_LORrank(self):
        return self.LORrank
    def get_ONrank(self):
        return self.ONrank
    def get_symbol(self):
        return self.symbol
    def get_factors(self):
        return self.factors
    def get_LORcontract(self):
        return LORcontract
    def get_ONcontract(self):
        return ONcontract
    
    def set_massDim(self, massDim):
        self.massDim = massDim
    def set_LORindices(self, LORindices):
        self.LORindices = LORindices
    def set_ONindices(self, ONindices):
        self.ONindices = ONindices
    def set_symbol(self, symbol):
        self.symbol = symbol
    def set_factors(self, factors):
        self.factors = factors
    def set_LORcontract(self, LORcontract):
        self.LORcontract = LORcontract
    def set_ONcontract(self, ONcontract):
        self.ONcontract = ONcontract
    #No set_LORrank() or set_ONrank() functions since LORrank and ONrank are determined by length of index lists
        
    #Returns a new field corresponding to derivative of field by increasing massDim and LORindex by one
    def derivative(self, mu = 'u'): 
        massDim_new = self.get_massDim() + 1
        #print(self.get_LORindices())
        LORindices_new = self.get_LORindices() + [mu]
        #print(LORindices_new)
        LORrank_new = self.get_LORrank() + 1
        fieldDeriv = field(massDim_new, LORindices_new, self.ONindices, LORrank_new, self.ONrank, 'D' + self.symbol)
        return fieldDeriv    
    
    #Returns tensor string showing full index structure of field object
    def makeTensorRep(self):
        tensor = self.symbol
        for i in self.LORindices:
            tensor = tensor + '_' + str(i)
        for j in self.ONindices:
            tensor = tensor + '^' + str(j)
        return tensor
    
    #Multiplies two fields, contracting along specified indices
    def contract(self, other, LORcontract, ONcontract):
        '''
        Calculates product/contraction of two fields; can be broken down recursively into elementary factors of field and 
        field derivative.
        INPUT
        self, other: field objects to be contracted along specified indices
        LORcontract: list of pairs (lists) of LOR indices (one from each field) to be contracted
        ONcontract: list of pairs (lists) of O(N) indices (one from each field) to be contracted
        
        OUTPUT
        new field object, prod, with factors stored in attribute prod.factors and contraction indices preserved 
        in prod.LORcontract and prod.ONcontract. 
        ''' 
        #Calculate massDim of product
        massDim_prod = self.get_massDim() + other.get_massDim()
        
        #Calculate LORindices, ONindices of product. [:] makes copy rather than mere reference/pointer, so that we don't alter attributes of factor fields. 
        LORindices_self = self.get_LORindices()[:]
        LORindices_other = other.get_LORindices()[:]

        ONindices_self = self.get_ONindices()[:]
        ONindices_other = other.get_ONindices()[:]
        
        print("LORindices_self: " + str(LORindices_self))
        print("LORindices_other: " + str(LORindices_other))
        
        #Standardize LORindex labels
        for i in range(len(LORindices_self)):
            LORindices_self[i] = 'u' + str(i)
        for i in range(len(LORindices_other)):
            LORindices_other[i] = 'v' + str(i)
        
        #Replace contracted LORindex labels with A's 
        if len(LORcontract) == 0: #check whether list of contracted LORindices lists is empty
            pass
        else: 
            for i in range(len(LORcontract)): 
                LORindices_self[LORcontract[i][0]] = 'S' + str(i)
                LORindices_other[LORcontract[i][1]] = 'S' + str(i)
    
        
        
        #Standardize ONindex labels
        for i in range(len(ONindices_self)):
            ONindices_self[i] = 'i' + str(i)
        for i in range(len(ONindices_other)):
            ONindices_other[i] = 'j' + str(i)
        
        #Replace contracted ONindex labels with B's
        if len(ONcontract) == 0: #check whether list of contracted ONindices lists is empty
            pass
        else:
            for i in range(len(ONcontract)): 
                ONindices_self[ONcontract[i][0]] = 'T' + str(i)
                ONindices_other[ONcontract[i][1]] = 'T' + str(i)
     
        self_contracted = field(self.get_massDim(), LORindices_self, ONindices_self, self.get_symbol())
        other_contracted = field(other.get_massDim(), LORindices_other, ONindices_other, other.get_symbol())
    
        #tensor_prod = '(' + self.tensorRep() + ')' + '(' + other.tensorRep() + ')'
    
        print("LORindices_self: " + str(LORindices_self))
        print("LORindices_other: " + str(LORindices_other))
        #remove contracted indices
        for i in range(len(LORcontract)):
            print("i: " + str(i))
            print("LORcontract[i][0]: " + str(LORcontract[i][0]) )
            LORindices_self.pop(LORcontract[i][0]-i)
            LORindices_other.pop(LORcontract[i][1]-i)
        for i in range(len(ONcontract)):
            ONindices_self.pop(ONcontract[i][0]-i)
            ONindices_other.pop(ONcontract[i][1]-i)
        
        LORindices_prod = LORindices_self +  LORindices_other
        ONindices_prod = ONindices_self + ONindices_other
        symbol_prod = '(' + '(' + self.get_symbol() + ')' + '(' + other.get_symbol() + ')' + ')'
        
        prod = field(massDim_prod, LORindices_prod, ONindices_prod, symbol_prod)
        prod.set_factors([self, other])
        prod.set_LORcontract(LORcontract)
        prod.set_ONcontract(ONcontract)
        
        return prod
        
        

In [134]:
A = field(5, ['u0', 'u1', 'u2'], ['i0', 'i1'], 'A')

In [135]:
B = field(3, ['v0', 'v1'], ['j0', 'j1'], 'B')

In [136]:
AB = field.contract(A, B, [[0, 1],[2, 0]], [[1, 0]])
#A.contract(B, [[0, 1],[2, 0]], [[1, 0]])
print(AB)

LORindices_self: ['u0', 'u1', 'u2']
LORindices_other: ['v0', 'v1']
LORindices_self: ['S0', 'u1', 'S1']
LORindices_other: ['S1', 'S0']
i: 0
LORcontract[i][0]: 0
i: 1
LORcontract[i][0]: 2
massDim: 8
LORindices: ['u1']
ONindices: ['i0', 'j1']
symbol: ((A)(B))
LORrank: 1
ONrank: 2
tensorRep: 


In [129]:
LORcontract = [[0, 1],[2, 0]]

In [130]:
LORindices_self = ['u0', 'u1', 'u2']

In [131]:
LORindices_self.pop(0)

'u0'

In [132]:
LORindices_self

['u1', 'u2']